In [ ]:
import pyarrow.feather as feather
from concurrent.futures import ThreadPoolExecutor
import pathlib, os, pickle, time, base64, uuid,sys
import numpy as np
from tqdm import tqdm

def load_pickle(thisPickle):
    with open(str(thisPickle), 'rb') as handle:
        batch_features = pickle.load(handle)
    return batch_features.reshape(-1, 4096)

def load_pickle_filesList(thisPickle):
    with open(str(thisPickle), 'rb') as handle:
        batch_elems = pickle.load(handle)
    return batch_elems

Let's first import our most recent checkpoint - a clustered set of 600,000 images.

In [ ]:
pickleFolder = pathlib.Path("faiss/600k")
feat_pickles = pickleFolder.rglob('features*.pickle')
list_of_pickles =  [str(p) for p in feat_pickles]

with ThreadPoolExecutor() as executor:
    features_list = list(tqdm(executor.map(load_pickle, list_of_pickles), total=len(list_of_pickles), desc="Loading Pickles"))

print(f"concatenating features")
features = np.concatenate(features_list, axis=0)
print(f"{features.shape[0]} features loaded")


We'll then cluster another 100,000 using the same API and the same settings to get the "real" group to which this image should belong

Then we'll implement a FAISS database of the first 600,000 and then use this to predict the destination of the additional 100,000 images:

Comparing the two we see that the "correct" group was predicted x% of the time
Getting the groups for all 700,000 took x seconds, while predicting the additional 100,000 groups took y seconds. Of course as the problem scales we may have issues with VRAM or similar, but for now x...